In [35]:
import pandas as pd

df = pd.read_csv("https://raw.githubusercontent.com/brobers0339/Musical_Impact_on_Mental_Health_Senior_Thesis/refs/heads/main/Music%26MentalHealthDataset.csv")

df_cleaned = df.drop(['Timestamp', 'Age', 'Primary streaming service', 'Exploratory', 'Permissions', 'BPM', 'Foreign languages'], axis=1)

df_cleaned['obs_count'] = range(1, len(df_cleaned) + 1)


In [26]:
print(df_cleaned.columns)

Index(['Hours per day', 'While working', 'Instrumentalist', 'Composer',
       'Fav genre', 'Frequency [Classical]', 'Frequency [Country]',
       'Frequency [EDM]', 'Frequency [Folk]', 'Frequency [Gospel]',
       'Frequency [Hip hop]', 'Frequency [Jazz]', 'Frequency [K pop]',
       'Frequency [Latin]', 'Frequency [Lofi]', 'Frequency [Metal]',
       'Frequency [Pop]', 'Frequency [R&B]', 'Frequency [Rap]',
       'Frequency [Rock]', 'Frequency [Video game music]', 'Anxiety',
       'Depression', 'Insomnia', 'OCD', 'Music effects', 'obs_count',
       'Effects', 'Effects_remapped'],
      dtype='object')


In [45]:
import statsmodels.api as sm
from statsmodels.miscmodels.ordinal_model import OrderedModel

df_cleaned['Effects_remapped'] = df_cleaned['Music effects'].map({
    'Improve' : 'Improved',
    'No effect' : 'No Effect',
    'Worsen' : 'Worsened',
    })
df_cleaned['Effects_remapped'] = df_cleaned['Effects_remapped'].fillna('Unknown')
df_cleaned['Effects_remapped_cat'] = df_cleaned['Effects_remapped'].map({
    'Improved' : 1,
    'No Effect' : 0,
    'Worsened' : -1,
    'Unknown' : 0
})
model = OrderedModel(
    df_cleaned['Effects_remapped_cat'],
    df_cleaned[['Anxiety', 'Depression', 'OCD', 'Insomnia', 'Frequency [Rap]']],
    distr='logit'
)

res = model.fit(method='bfgs')
print(res.summary())

ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).